In [ ]:
### TODO
# use the order flow intensity function in the Gold example in the BTC example

In [ ]:
# import all necessary research utilities
from research_utilities import *
%matplotlib inline


# import libraries
from datetime import date
import pandas as pd
pd.options.display.max_columns = None 
import numpy as np

# charting libraries & set style
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn")

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# import our own utilities
from macro_model import *

# configs
today = date.today()
data_start_date = "2005-01-01"
analysis_start_date = "2020-07-01"
end_date = today

In [ ]:
df_trades = pd.read_parquet("/Users/jakemoore/Desktop/BITMEX-XBTUSD-trades-1612302410.parquet")

    df_l2 = pd.read_parquet("/Users/jakemoore/Desktop/BINANCE-BTC-USDT-l2_book-1612295089.parquet")

In [ ]:
df_trades["timestamp"] = pd.to_datetime(df_trades["timestamp"], unit="s")
df_trades["receipt_timestamp"] = pd.to_datetime(df_trades["receipt_timestamp"], unit="s")

In [ ]:
df_l2["timestamp"] = pd.to_datetime(df_l2["timestamp"], unit="s")

In [ ]:
df_l2["receipt_timestamp"] = pd.to_datetime(df_l2["receipt_timestamp"], unit="s")

In [ ]:
df_trades.tail()

In [ ]:
df_l2.set_index("timestamp", inplace=True)

In [ ]:
# read trades data

In [ ]:
trades = pd.read_parquet("/Users/jakemoore/Desktop/BINANCE-BTC-USDT-trades-1612295089.parquet")

In [ ]:
trades["timestamp"] = pd.to_datetime(trades["timestamp"], unit="s")

In [ ]:
trades["receipt_timestamp"] = pd.to_datetime(trades["receipt_timestamp"], unit="s")

In [ ]:
trades = trades.set_index(trades["timestamp"])


In [ ]:
trades["direction"] = np.where(trades["side"]=='buy', 1, -1)
trades["direction_size"] = trades["direction"] * trades["amount"] 

In [ ]:
trades.head()

Research - pockets of volatility and liquidity 
Invast gold spread
Go with large flow or just flow when YP combines with breakout 
How get all the trades
How reliable is invast spread?

Research - my pricing engine


### Research - pockets of volatility and liquidity 
Invast gold spread.

Go with large flow or just flow when YP combines with breakout.

How get all the trades?

How reliable is invast spread?

Where is simulator? 

BREAKOUT and flow combined 

### Research - my pricing engine


In [ ]:
# test = trades.resample("1s").agg({'x':'sum','y':'mean','z':'last'})


In [ ]:
# trades_10ms = trades.resample("10ms").last().ffill()

## Flow trading 

In [ ]:
def lag_price(
    data, column="price", windows=[-100, -500, -1000,-3000, -6000]): # the unit is the resample period i.e. 10ms
    for window in windows:
        data["lagged_price_{}0_ms".format(window)] = (
            data[column].shift(window))
    return


In [ ]:
tr = trades.drop_duplicates(subset=['timestamp'], keep='last')
# trades = trades.resample("10ms").last().ffill()

In [ ]:
lag_price(tr)

In [ ]:
tr.head()

In [ ]:
tr["amount_1s_flow"] = tr['direction_size'].rolling("1s").sum() 
tr["amount_5s_flow"] = tr['direction_size'].rolling("5s").sum() 
tr["amount_60s_flow"] = tr['direction_size'].rolling("60s").sum() 

tr["amount_5m_flow"] = tr['direction_size'].rolling("5t").sum() 


In [ ]:
tr["1s_yield_flow"] = ((tr["lagged_price_-1000_ms"] / tr["price"])-1) * tr["amount_5s_flow"]
tr["30s_yield_flow"] = ((tr["lagged_price_-30000_ms"] / tr["price"])-1) * tr["amount_5s_flow"]


tr["1s_yield"] = ((tr["lagged_price_-1000_ms"] / tr["price"])-1) * tr["direction"]
tr["5s_yield"] = ((tr["lagged_price_-5000_ms"] / tr["price"])-1) * tr["direction"]

tr["30s_yield"] = ((tr["lagged_price_-30000_ms"] / tr["price"])-1) * tr["direction"]

    

In [ ]:
## Need to change this to trade flow not trade size

df_adverse = tr
# df_adverse = tr.loc[tr["amount"] >= 0.01]
# df_adverse = tr.loc[tr["amount_5s_flow"] >= 0]
df_adverse = df_adverse.loc[df_adverse["1s_yield_flow"] > 0.005]
    


In [ ]:
tr.head()

In [ ]:
df_adverse.describe()

In [ ]:
df_adverse["sum"] = df_adverse["1s_yield_flow"].cumsum().plot(title="when amount > 0.02 and 1s yield is > 0.0002");    
# plt.savefig("/Users/jakemoore/Desktop/research folder/adverse_selction_when amount > 0.02 and 1s yield is > 0.0002.png")

### Trade flow - going with the flow

In [ ]:
tr["flow_signal"] = np.where(tr["amount_5s_flow"] > 0, 1, -1)
tr["flow_signal"] = tr["flow_signal"].shift(1)
tr["flow_pnl"] = tr["flow_signal"] * tr["price"].pct_change()

In [ ]:
tr["flow_signal"].plot()

In [ ]:
tr["flow_pnl"].cumsum().plot()

In [ ]:
tr['direction_size_100ms'] = tr["direction_size"].rolling("100ms").sum()

In [ ]:
big_fish = tr[np.abs(tr["direction_size_100ms"]) > 0.25]

In [ ]:
big_fish["flow_pnl"].cumsum().plot()

In [ ]:
big_fish["flow_pnl"].mean()

### Think Gold Flow

In [ ]:
xau_flow = pd.read_csv("/Users/jakemoore/Desktop/XAUThinkW01-04.csv")
xau_flow["UnixTime"] = pd.to_datetime(xau_flow["UnixTime"]/1000, unit="s")
xau_flow = xau_flow.set_index(xau_flow["UnixTime"])

In [ ]:
xau_flow.tail()

In [ ]:
# xau_flow = xau_flow.between_time("07:00", "17:00")
xau_flow["direction"] = np.where(xau_flow["side"] == 'OFFER', 1, -1)

In [ ]:
xau_flow["direction_flow"] = xau_flow["direction"] * xau_flow["trade_quantity"]

In [ ]:
xau_flow["cum_flow"] = xau_flow["direction_flow"].cumsum()

In [ ]:
xau_flow[["cum_flow", "trade_price"]].resample("60t").last().plot(subplots=True);

In [ ]:
xau_flow["5min_flow"] = xau_flow["direction_flow"].rolling("5t").sum()
xau_flow["15min_flow"] = xau_flow["direction_flow"].rolling("15t").sum()
xau_flow["30min_flow"] = xau_flow["direction_flow"].rolling("30t").sum()


# simple flow
xau_flow["flow_signal"] = np.where(xau_flow["5min_flow"] < 0, 1, -1)
xau_flow["flow_signal"] = xau_flow["flow_signal"].shift(1)
xau_flow["flow_pnl"] = xau_flow["flow_signal"] * xau_flow["trade_price"].diff()
xau_flow["flow_pnl_cum"] = xau_flow["flow_pnl"].cumsum()



In [ ]:
xau_flow["flow_pnl"].cumsum().resample("5t").last().plot(subplots=True);

In [ ]:
# xau_flow["5m_ma"] = xau_flow["trade_price"].rolling("5t").mean()
# xau_flow["5m_ma_signal"] = np.where(xau_flow["trade_price"] > xau_flow["5m_ma"], -1, 1)
# xau_flow["5m_ma_signal"] = xau_flow["5m_ma_signal"].shift(1)
# xau_flow["5m_ma_pnl"] = xau_flow["5m_ma_signal"] * xau_flow["trade_price"].diff()

In [ ]:
trades=11
cost_per_trade = 0.2
cost=trades*cost_per_trade
cost

In [ ]:
xau_flow["flow_pnl"]["2021-01-08 14:00:00":"2021-01-08 15:00:00"].cumsum().resample("1t").last().plot(subplots=True);

In [ ]:
### TO INVESTIGATE
# pnl when trend is positive and negagtive

In [ ]:
xau_flow[["flow_signal", "flow_pnl_cum"]]["2021-01-08 14:00:00":"2021-01-08 15:00:00"].plot(subplots=True)

In [ ]:
xau_flow['signal_change'] = np.where(xau_flow["flow_signal"] != xau_flow["flow_signal"].shift(1), 1, np.nan)

In [ ]:
xau_flow.describe()

In [ ]:
2465*.2

In [ ]:
xau_flow['signal_change'].count()

In [ ]:
xau_flow['flow_pnl'].sum()

In [ ]:
small_fry = xau_flow[xau_flow["trade_quantity"] < 3]
big_fish = xau_flow[xau_flow["trade_quantity"] > 100]

In [ ]:
small_fry_yield = small_fry['flow_pnl'].sum() / small_fry['signal_change'].count()
small_fry_yield

In [ ]:
big_fish_yield = big_fish['flow_pnl'].sum() / big_fish['signal_change'].count()
big_fish_yield

In [ ]:
small_fry.describe()

In [ ]:
big_fish.describe()

In [ ]:
# xau_flow['big_flow_column', 'small_flow_column', 'sum_big_flow', 'sum_small_flow']
# market orders for big flow 

In [ ]:
big_fish_yield = big_fish['flow_pnl'].cumsum().plot()

In [ ]:
total_flow_pnl = np.concatenate((small_fry["flow_pnl"], -big_fish['flow_pnl']))

In [ ]:
total = pd.DataFrame(total_flow_pnl)


In [ ]:
total.cumsum().plot()

In [ ]:
# look at the two sizes, are there any predictive relationships?
xau_flow["small_fry_pnl"] = np.where(xau_flow["trade_quantity"] < 3, xau_flow["flow_pnl"], np.nan)
xau_flow["big_fish_pnl"] = np.where(xau_flow["trade_quantity"] >100, xau_flow["flow_pnl"]*(-1), np.nan)



In [ ]:
xau_flow["big_fish_pnl"]["2021-01-06 08:45:00":"2021-01-09 15:00:00"].rolling("5t").count().plot();

In [ ]:
# just take the signals 

xau_flow["big_fish_pnl_filtered"] = np.where(xau_flow["big_fish_pnl"].shift(1).rolling("5t").count()>10, xau_flow["big_fish_pnl"], np.nan)

In [ ]:
xau_flow["big_fish_pnl_filtered"].describe()

In [ ]:
xau_flow["big_fish_pnl_filtered"]["2021-01-04 08:45:00":"2021-01-08 20:00:00"].cumsum().plot();

In [ ]:
### RESEARCH
# Order flow intensity 
# More orders = more movement?
# How switch regimes 
# Auto select big flow 

In [ ]:
xau_flow["trade_price"].plot();

In [ ]:
xau_flow["big_fish_pnl_filtered"]

In [ ]:
xau_flow.head()